# Assignment 2

## Roll number: 


### Instructions
 * Fill in the roll-number in the cell above.
 * Code must be submitted in Python in jupyter notebooks. We highly recommend using anaconda/miniconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 *  Most of the questions require you to **code your own functions** unless there is a need to call in the abilities of the mentioned libraries, such as Visualisation from Open3D. Make sure your code is modular since you will be reusing them for future assignments. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 *  All the representations are expected to be in a right-hand coordinate system.
<!--  * Answer to the descriptive questions should be answered in your own words. Copy-paste answers will lead to penalty. -->
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.
 * Commit the notebooks in the repo and any other results files under the result folder in the GitHub Classroom repo. 
 * This is a group assignment. Discussions are encouraged but any sharing of code among different teams will be penalized. 

## SECTION 1: ICP with SVD

### 1.1 Perform Procrustes alignmenton two point clouds with (given) known correspondences. (5 Points)

Let X be your point cloud observed from the initial pose P1. You then transform it to a new pose P2. Now you wish to apply ICP to recover transformation between (X & P1) and (X & P2).

Use toothless.ply point cloud and perform the alignment between the two point clouds using procrustes alignment. Your task is to write a function that takes two point clouds as input wherein the corresponding points between the two point clouds are located at the same index and returns the transformation matrix between them. Compute the alignment error after aligning the two point clouds.

<b>Use root mean squared error (RSME) as the alignment error metric.</b>

Make sure your code is modular as we will use this function in the next sub-part.

We will again use our own getTransform function to generate the 4x4 Transformation matrix to transform the pointcloud to P2, so make sure your code works for any general Transformation matrix

In [ ]:
def getTransform():
    pass

In [ ]:
T = getTransform()
def icp_known_correspondences(T):
    #### YOUR CODE HERE ####
    pass

### 1.2 Implement ICP algorithm with unknown correspondences. (5 Points)

Your task is to write a function that implements ICP and takes two point clouds as input wherein the correspondances are unknown. Visualize the pointclouds and plot their individual coordinate frames as you perform ICP over them. Compute the alignment error in each iteration. 

Refer to Shubodh's notes to compute correspondences: https://saishubodh.notion.site/Mobile-Robotics-Navigating-from-Theory-to-Application-0b65a9c20edd4081978f4ffad917febb?p=a25686ce1a11409d838d47bcac43ab4b&pm=s#bb9aaf2e316b4db3b399df1742f0444c


In [ ]:
P = getTransform()
Q = getTransform()

def icp_unknown_correspondences(P, Q):
    #### YOUR CODE HERE ####
    pass

## SECTION 2: ICP with Lie Groups

### 2.1 Predict the Transformation matrix between 2 point clouds with known correspondences (15 Points)
Perform the same task as 1.1 using Lie Group Optimization from scratch to predict the transformation between the 2 point clouds.

Refer: https://saishubodh.notion.site/Mobile-Robotics-Navigating-from-Theory-to-Application-0b65a9c20edd4081978f4ffad917febb?p=ee55fe5689794693910ab7861bef067b&pm=s#7b82d84766a84b63b91d859579e4886b


In [ ]:
T = getTransform()
def icp_with_lie(T):
    #### YOUR CODE HERE ####
    pass

## SECTION 3: Pose Graph Optimization with G2O
### Objective (5 Points)
A robot is travelling in a oval trajectory. It is equipped with wheel odometry for odometry information and RGBD sensors for loop closure information. Due to noise in wheel odometry it generates a noisy estimate of the trajectory. Our task is to use loop closure pairs to correct the drift.

We pose this problem as a pose graph optimization problem. In our graph, poses are the vertices and constraints are the edges. 

References:

1.) Class notes: https://saishubodh.notion.site/G2O-Edge-Types-d9f9ff63c77c4ceeb84b1e49085004e3

2.) Cyrill Stachniss lecture: https://www.youtube.com/watch?v=uHbRKvD8TWg 

### Given: 
In practical scenarios, we'd obtain the following from our sensors after some post-processing:

1. Initial position
2. Odometry Contraints/Edges: This "edge" information tells us relative transformation between two nodes. These two nodes are consecutive in the case of Odometry but not in the case of Loop Closure (next point).
3. Loop Closure Contraints/Edges: Remember that while optimizing, you have another kind of "anchor" edge as you've seen in 1. solved example.

You have been given a text file named `edges.txt` (in `data/`) which has all the above 3 and it follows G2O's format (as explained in class, [link here](https://saishubodh.notion.site/G2O-Edge-Types-d9f9ff63c77c4ceeb84b1e49085004e3) ). The ground truth is `gt.txt`.

Install g2o as mentioned in `g2o.ipynb` and optimise `edges.txt`

In [ ]:
import math
import numpy as np
from matplotlib import pyplot as plt
import os

In [ ]:
#Given Helper Functions
def readVertex(fileName):
    f = open(fileName, 'r')
    A = f.readlines()
    f.close()

    x_arr = []
    y_arr = []
    theta_arr = []

    for line in A:
        if "VERTEX_SE2" in line:
            (ver, ind, x, y, theta) = line.split()
            x_arr.append(float(x))
            y_arr.append(float(y))
            theta_arr.append(float(theta.rstrip('\n')))

    return np.array([x_arr, y_arr, theta_arr])

def readEdge(fileName):
    f = open(fileName, 'r')
    A = f.readlines()
    f.close()

    ind1_arr = []
    ind2_arr = []
    del_x = []
    del_y = []
    del_theta = []

    for line in A:
        if "EDGE_SE2" in line:
            (edge, ind1, ind2, dx, dy, dtheta, _, _, _, _, _, _) = line.split()
            ind1_arr.append(int(ind1))
            ind2_arr.append(int(ind2))
            del_x.append(float(dx))
            del_y.append(float(dy))
            del_theta.append(float(dtheta))

    return (np.array( ind1_arr), np.array(ind2_arr), np.array(del_x), np.array(del_y), np.array(del_theta))

def draw(X, Y, THETA):
    ax = plt.subplot(111)
    ax.plot(X, Y, 'ro')
    plt.plot(X, Y, 'c-')

    for i in range(len(THETA)):
        x2 = 0.25*math.cos(THETA[i]) + X[i]
        y2 = 0.25*math.sin(THETA[i]) + Y[i]
        plt.plot([X[i], x2], [Y[i], y2], 'g->')

    plt.show()

In [ ]:
# Ground Truth Data
input_file = "./data/gt.txt"
gt_vertex = readVertex(input_file)
gt_edges = readEdge(input_file)
print(gt_vertex.shape)
draw(gt_vertex[0],gt_vertex[1],gt_vertex[2])

In [ ]:
### YOUR CODE HERE ###
#code to generate the g2o file for edges.txt
input_file = "./data/edges.txt"
output_file = "./data/edges.g2o"

out = open(output_file, "w")

init = readVertex(input_file)
x = np.zeros(120)
y = np.zeros(120)
theta = np.zeros(120)
x[0] = init[0]
y[0] = init[1]
theta[0] = init[2]

ind1, ind2, dx, dy, dtheta = readEdge(input_file)
info_x_y_theta = "500.0 0.0 0.0 500.0 0.0 500.0"

print(len(ind1))

for i in range(len(x)-1):
    # USE MODEL TO CALCULATE VERTICES AND PROCEED
    # WAIT FOR MODEL TO BE GIVEN
    pass 

# for i in range(len(x)):
#     out.write("VERTEX_SE2" + " " + str(i) + " " + str(x[i]) + " " + str(y[i]) + " " + str(theta[i]) + "\n")

# for i in range(len(ind1)):
#     out.write("EDGE_SE2" + " " + str(ind1[i]) + " " + str(ind2[i]) + " " + str(dx[i]) + " " + str(dy[i]) + " " + str(dtheta[i]) + " " + info_x_y_theta + "\n")


# out.write("FIX 0"+"\n")

# print(np.size(x))
# draw(x,y,theta)

In [ ]:
#code to optimise this data with g2o
def optimize():
    cmd = "g2o -o ./data/optimal.g2o ./data/edges.g2o"
    os.system(cmd)
optimize()

In [ ]:
# Optimised Data
input_file = "./data/optimal.g2o"
gt_vertex = readVertex(input_file)
gt_edges = readEdge(input_file)
print(gt_vertex.shape)
draw(gt_vertex[0],gt_vertex[1],gt_vertex[2])

In [ ]:
#to view in g2o_viewer

os.system("g2o_viewer ./data/optimal.g2o")
os.system("g2o_viewer ./data/gt.g2o")

### Evo (10 Points)
We need a measure of how good the trajectory is. The error/loss used earlier doesn't tell us much about how the trajectory differs from the ground truth. Here, we try to do just this - compute error metrics. Rather than computing these from scratch, we will just Evo - https://github.com/MichaelGrupp/evo/.

Look at the absolute pose error (APE) and relative pose error (RPE). What do they capture and how are they calculated (descriptive answer)? How do these metrics differ in methodology? Can we determine if the error is more along the x/y axis?

Answer the above questions and report errors for the obtained trajectory.

In [ ]:
# Commands to convert to Kitti

# python3 ./misc/g2o_to_kitti.py ./data/gt.g2o ./data/gt.kitti

# python3 ./misc/g2o_to_kitti.py ./data/optimal.g2o ./data/optimal.kitti

# python3 ./misc/g2o_to_kitti.py ./data/edges.g2o ./data/edges.kitti

In [ ]:
# Commands to generate RPE Error Graphs
# evo_rpe kitti gt.kitti opt.kitti -v --plot --plot_mode xy

In [ ]:
# Commands to generate APE Error Graphs
# evo_ape kitti gt.kitti opt.kitti -v --plot --plot_mode xy

In [ ]:
# Commands to generate TRAJ Error Graphs
# evo_traj kitti gt.kitti opt.kitti -v --plot --plot_mode xy